In [5]:
import itertools
from collections import defaultdict, Counter

# 텍스트 데이터 (예제)
text = """
이것은 예제 문장입니다. 예제 문장을 가지고 N-gram과 마르코프 체인을 학습합니다.
여러개의 단어가 주어지면 최적의 단어 순서 배열을 찾아야 합니다.
이를 위해 다양한 방법을 사용할 수 있습니다.
"""

# 전처리 및 토큰화
tokens = text.split()

# N-gram 모델 생성 (여기서는 Trigram 예제)
trigrams = [(tokens[i], tokens[i+1], tokens[i+2]) for i in range(len(tokens)-2)]
trigram_freq = Counter(trigrams)

# 마르코프 체인 기반 전이 확률 테이블 생성
transitions = defaultdict(lambda: defaultdict(int))
for (w1, w2, w3) in trigrams:
    transitions[(w1, w2)][w3] += 1

# 전이 확률 계산
for prev_words, next_words in transitions.items():
    total = float(sum(next_words.values()))
    for word in next_words:
        next_words[word] /= total

# 최적의 단어 순서 배열 선택 함수
def score_sequence(sequence, transitions):
    score = 0.0
    for i in range(len(sequence) - 2):
        w1, w2, w3 = sequence[i], sequence[i+1], sequence[i+2]
        if (w1, w2) in transitions and w3 in transitions[(w1, w2)]:
            score += transitions[(w1, w2)][w3]
        else:
            score -= 1  # 낮은 확률로 패널티 부여
    return score

def find_best_sequence(words, transitions):
    best_sequence = None
    best_score = float('-inf')
    for perm in itertools.permutations(words):
        score = score_sequence(perm, transitions)
        if score > best_score:
            best_score = score
            best_sequence = perm
    return best_sequence

# 주어진 단어들
words = ["예제", "최적의", "단어", "순서", "배열", "찾기"]

# 최적의 단어 순서 배열 찾기
best_sequence = find_best_sequence(words, transitions)
print("최적의 단어 순서 배열:", best_sequence)


최적의 단어 순서 배열: ('예제', '최적의', '단어', '순서', '배열', '찾기')


In [4]:
import itertools
from collections import defaultdict, Counter

# 예시 데이터 준비
documents = [
    ["기술", "안전", "분석", "데이터"],
    ["공정", "관리", "품질", "생산"],
    ["안전", "기술", "분석", "공정"],
    ["데이터", "관리", "분석", "품질"]
]

# 모든 가능한 단어 조합 생성
def generate_sequences(documents):
    sequences = []
    for terms in documents:
        for i in range(1, len(terms)):
            for j in range(len(terms) - i + 1):
                seq = terms[j:j+i+1]
                sequences.append(seq)
    return sequences

sequences = generate_sequences(documents)

# N-gram 모델 생성
n = 2  # 예를 들어 Bigram을 사용
ngrams = defaultdict(Counter)

for sequence in sequences:
    for i in range(len(sequence) - n + 1):
        gram = tuple(sequence[i:i+n-1])
        next_word = sequence[i + n - 1]
        ngrams[gram][next_word] += 1

# 단어 배치 확률 계산 함수
def calculate_sequence_probability(sequence, ngrams):
    prob = 1.0
    for i in range(len(sequence) - 1):
        current_gram = tuple(sequence[i:i+n-1])
        next_word = sequence[i + n - 1]
        next_word_count = ngrams[current_gram][next_word]
        total_count = sum(ngrams[current_gram].values())
        prob *= next_word_count / total_count if total_count > 0 else 0
    return prob

# 최적의 단어 배치 찾기 함수
def find_best_sequence(words, ngrams):
    best_sequence = None
    best_prob = 0.0
    for perm in itertools.permutations(words):
        prob = calculate_sequence_probability(perm, ngrams)
        if prob > best_prob:
            best_prob = prob
            best_sequence = perm
    return best_sequence, best_prob

# 예측 예제
input_words = ["기술", "데이터", "안전", "분석"]
best_sequence, best_prob = find_best_sequence(input_words, ngrams)

print("Best sequence:", best_sequence)
print("Probability:", best_prob)


Best sequence: ('기술', '안전', '분석', '데이터')
Probability: 0.078125


In [5]:
from konlpy.tag import Okt
from soynlp.word import WordExtractor
from transformers import pipeline

# 한국어 형태소 분석기 초기화
okt = Okt()

# 사전 훈련된 한국어 언어 모델 로드 (예: KoBERT)
unmasker = pipeline('fill-mask', model='monologg/kobert')

def is_valid_combination(noun1, noun2):
    # 두 명사를 결합한 문장을 생성
    sentence = f"{noun1} {noun2}는"
    masked_sentence = sentence.replace(noun2, '[MASK]')
    
    # 언어 모델을 사용하여 [MASK] 위치의 단어 예측
    result = unmasker(masked_sentence)
    
    # 가장 높은 확률의 단어가 noun2와 일치하는지 확인
    for prediction in result:
        if prediction['token_str'] == noun2:
            return True
    return False

# 테스트할 명사 리스트
nouns = ['작동', '계획']

# 모든 조합에 대해 올바른 순서인지 판단
for i in range(len(nouns)):
    for j in range(len(nouns)):
        if i != j:
            noun1 = nouns[i]
            noun2 = nouns[j]
            if is_valid_combination(noun1, noun2):
                print(f"'{noun1} {noun2}'는 올바른 조합입니다.")
            else:
                print(f"'{noun1} {noun2}'는 올바르지 않은 조합입니다.")

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [1]:
from konlpy.tag import Okt
from transformers import pipeline

# 한국어 형태소 분석기 초기화
okt = Okt()

# 사전 훈련된 한국어 언어 모델 로드 (예: KoBERT)
unmasker = pipeline('fill-mask', model='monologg/kobert')

def is_valid_combination(noun1, noun2):
    # 두 명사를 결합한 문장을 생성
    sentence = f"{noun1} {noun2}는"
    masked_sentence = sentence.replace(noun2, '[MASK]')
    
    # 언어 모델을 사용하여 [MASK] 위치의 단어 예측
    result = unmasker(masked_sentence)
    
    # 가장 높은 확률의 단어가 noun2와 일치하는지 확인
    for prediction in result:
        if prediction['token_str'] == noun2:
            return True
    return False

# 테스트할 명사 리스트
nouns = ['작동', '계획']

# 모든 조합에 대해 올바른 순서인지 판단
for i in range(len(nouns)):
    for j in range(len(nouns)):
        if i != j:
            noun1 = nouns[i]
            noun2 = nouns[j]
            if is_valid_combination(noun1, noun2):
                print(f"'{noun1} {noun2}'는 올바른 조합입니다.")
            else:
                print(f"'{noun1} {noun2}'는 올바르지 않은 조합입니다.")

c:\Users\PC\anaconda3\envs\interX_work\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\PC\anaconda3\envs\interX_work\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate dev

'작동 계획'는 올바르지 않은 조합입니다.
'계획 작동'는 올바르지 않은 조합입니다.


In [8]:
!pip install torch torchvision torchaudio

  Using cached torchvision-0.18.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.3.1-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.2 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ------------------------ --------------- 1.4/2.4 MB 45.6 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 30.2 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForMaskedLM.from_pretrained('monologg/kobert')

def is_valid_combination(noun1, noun2):
    # 두 명사를 결합한 문장을 생성
    sentence = f"{noun1} {noun2}는"
    masked_sentence = sentence.replace(noun2, '[MASK]')
    
    # 토크나이저를 사용하여 문장 토큰화
    inputs = tokenizer(masked_sentence, return_tensors='pt')
    
    # 모델을 사용하여 [MASK] 위치의 단어 예측
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits

    # [MASK] 토큰의 인덱스 추출
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    # [MASK] 토큰 위치의 단어 예측
    mask_token_logits = predictions[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    # 가장 높은 확률의 단어가 noun2와 일치하는지 확인
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_5_tokens)
    if noun2 in predicted_tokens:
        return True
    return False

# 테스트할 명사 리스트
nouns = ['작동', '계획']

# 모든 조합에 대해 올바른 순서인지 판단
for i in range(len(nouns)):
    for j in range(len(nouns)):
        if i != j:
            noun1 = nouns[i]
            noun2 = nouns[j]
            if is_valid_combination(noun1, noun2):
                print(f"'{noun1} {noun2}'는 올바른 조합입니다.")
            else:
                print(f"'{noun1} {noun2}'는 올바르지 않은 조합입니다.")


Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'작동 계획'는 올바르지 않은 조합입니다.
'계획 작동'는 올바르지 않은 조합입니다.


In [3]:
from konlpy.tag import Okt
from collections import Counter
import re

# 형태소 분석기 초기화
okt = Okt()

# 예제 텍스트 데이터 (여기서는 간단한 예제만 사용하지만, 실제로는 대규모 코퍼스를 사용해야 합니다)
text_data = """
계획 작동은 중요한 과정입니다. 작동 계획이 제대로 이루어져야 합니다.
계획을 작동하는 방법은 여러 가지가 있습니다. 좋은 계획은 성공적인 작동으로 이어집니다.
"""

# 명사 추출 함수
def extract_nouns(text):
    sentences = text.split('\n')
    nouns = []
    for sentence in sentences:
        tokens = okt.pos(sentence, norm=True, stem=True)
        nouns.extend([word for word, pos in tokens if pos == 'Noun'])
    return nouns

# 명사 쌍 추출 함수
def extract_noun_pairs(nouns):
    pairs = []
    for i in range(len(nouns) - 1):
        pairs.append((nouns[i], nouns[i+1]))
    return pairs

# 명사 추출
nouns = extract_nouns(text_data)

# 명사 쌍 추출
noun_pairs = extract_noun_pairs(nouns)

# 명사 쌍 빈도 계산
pair_counts = Counter(noun_pairs)

# 일정 빈도 이상의 명사 쌍만 "올바른 조합"으로 간주
threshold = 1
valid_pairs = {pair: count for pair, count in pair_counts.items() if count > threshold}

# 결과 출력
print("올바른 명사 조합:")
for pair, count in valid_pairs.items():
    print(f"{pair[0]} {pair[1]}: {count}회")

# 테스트할 명사 리스트
nouns_to_test = ['작동', '계획']

# 테스트 함수
def is_valid_combination(noun1, noun2):
    return (noun1, noun2) in valid_pairs

# 모든 조합에 대해 올바른 순서인지 판단
for i in range(len(nouns_to_test)):
    for j in range(len(nouns_to_test)):
        if i != j:
            noun1 = nouns_to_test[i]
            noun2 = nouns_to_test[j]
            if is_valid_combination(noun1, noun2):
                print(f"'{noun1} {noun2}'는 올바른 조합입니다.")
            else:
                print(f"'{noun1} {noun2}'는 올바르지 않은 조합입니다.")


올바른 명사 조합:
계획 작동: 2회
'작동 계획'는 올바르지 않은 조합입니다.
'계획 작동'는 올바른 조합입니다.


In [4]:
from konlpy.tag import Okt
from transformers import BertTokenizer, BertForMaskedLM
import torch
import itertools

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForMaskedLM.from_pretrained('monologg/kobert')

# 형태소 분석기 초기화
okt = Okt()

# 명사 조합의 적절성을 평가하는 함수
def is_valid_combination(nouns):
    sentence = " ".join(nouns) + "는"
    valid = True
    for i in range(len(nouns)):
        masked_sentence = sentence.replace(nouns[i], '[MASK]')
        inputs = tokenizer(masked_sentence, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
        predictions = outputs.logits
        mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        mask_token_logits = predictions[0, mask_token_index, :]
        top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
        predicted_tokens = tokenizer.convert_ids_to_tokens(top_5_tokens)
        if nouns[i] not in predicted_tokens:
            valid = False
            break
    return valid

# 테스트할 명사 리스트
nouns_to_test = ['작동', '계획', '설계', '운영', '안전', '검토']

# 모든 조합에 대해 올바른 순서인지 판단 (최대 6개 명사까지)
print("테스트할 명사 조합:")
for length in range(2, 7):
    for combination in itertools.combinations(nouns_to_test, length):
        if is_valid_combination(combination):
            print(f"'{combination}'는 올바른 조합입니다.")
        else:
            print(f"'{combination}'는 올바르지 않은 조합입니다.")


Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


테스트할 명사 조합:
'('작동', '계획')'는 올바르지 않은 조합입니다.
'('작동', '설계')'는 올바르지 않은 조합입니다.
'('작동', '운영')'는 올바르지 않은 조합입니다.
'('작동', '안전')'는 올바르지 않은 조합입니다.
'('작동', '검토')'는 올바르지 않은 조합입니다.
'('계획', '설계')'는 올바르지 않은 조합입니다.
'('계획', '운영')'는 올바르지 않은 조합입니다.
'('계획', '안전')'는 올바르지 않은 조합입니다.
'('계획', '검토')'는 올바르지 않은 조합입니다.
'('설계', '운영')'는 올바르지 않은 조합입니다.
'('설계', '안전')'는 올바르지 않은 조합입니다.
'('설계', '검토')'는 올바르지 않은 조합입니다.
'('운영', '안전')'는 올바르지 않은 조합입니다.
'('운영', '검토')'는 올바르지 않은 조합입니다.
'('안전', '검토')'는 올바르지 않은 조합입니다.
'('작동', '계획', '설계')'는 올바르지 않은 조합입니다.
'('작동', '계획', '운영')'는 올바르지 않은 조합입니다.
'('작동', '계획', '안전')'는 올바르지 않은 조합입니다.
'('작동', '계획', '검토')'는 올바르지 않은 조합입니다.
'('작동', '설계', '운영')'는 올바르지 않은 조합입니다.
'('작동', '설계', '안전')'는 올바르지 않은 조합입니다.
'('작동', '설계', '검토')'는 올바르지 않은 조합입니다.
'('작동', '운영', '안전')'는 올바르지 않은 조합입니다.
'('작동', '운영', '검토')'는 올바르지 않은 조합입니다.
'('작동', '안전', '검토')'는 올바르지 않은 조합입니다.
'('계획', '설계', '운영')'는 올바르지 않은 조합입니다.
'('계획', '설계', '안전')'는 올바르지 않은 조합입니다.
'('계획', '설계', '검토')'는 올바르지 않은 조합입니다.
'('계획', '운영', '안전')'는 올바르지 않은 조합입니다.
'('계획

In [5]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 문장에서 단어의 품사를 분석하는 함수
def analyze_pos(text, target_word):
    pos_tags = okt.pos(text)
    results = []
    for word, pos in pos_tags:
        if word == target_word:
            results.append((word, pos))
    return results

# 테스트할 문장과 단어
sentence = "계획을 잘 세워야 합니다. 작동하는 기계를 점검하세요. 계획된 작업을 수행하십시오."
target_words = ["계획", "작동"]

# 각 단어가 문장에서 어떤 품사로 사용되었는지 분석
for word in target_words:
    pos_analysis = analyze_pos(sentence, word)
    print(f"단어 '{word}'의 품사 분석 결과:")
    for analysis in pos_analysis:
        print(f" - {analysis[0]}: {analysis[1]}")


단어 '계획'의 품사 분석 결과:
 - 계획: Noun
 - 계획: Noun
단어 '작동'의 품사 분석 결과:
 - 작동: Noun
